In [2]:
import glob, os, sys
import pandas as pd
import geopandas as gpd

In [3]:
REGION = 'CAS'
dir_strips_toa = f'/adapt/nobackup/projects/3sl/data/VHR/{REGION}/M1BS'
dir_strips_tap = '/adapt/nobackup/projects/3sl/labels/landcover/tappan'

In [4]:
# Get strips used for tappan squares 
list_strips_tap = ['_'.join( os.path.basename(f).replace("_mask_segs_reclassified.tif","").split("_")[1:] ) for f in glob.glob(dir_strips_tap + "/*.tif")]

In [5]:
# Find the TOAs for these strips
list_toa = glob.glob(dir_strips_toa + "/*toa.tif")

In [6]:
list_strips_tap_toa = []
for strip in list_strips_tap:
    for toa in list_toa:
        if strip in toa:
            #print(toa)
            list_strips_tap_toa.append(toa)

In [7]:
len(list_strips_tap)

38

In [8]:
len(list_strips_tap_toa)

38

### We need CCDC for these

In [9]:
print("We need CCDC for these TOAs...")
print(f"Here are the paths to the TOAs matching the strips associated with Tappan squares in {REGION}:\n")
print(f"{[print(f) for f in list_strips_tap_toa]}")

We need CCDC for these TOAs...
Here are the paths to the TOAs matching the strips associated with Tappan squares in CAS:

/adapt/nobackup/projects/3sl/data/VHR/CAS/M1BS/WV03_20170226_M1BS_104001002802FF00-toa.tif
/adapt/nobackup/projects/3sl/data/VHR/CAS/M1BS/WV02_20170126_M1BS_1030010062B12000-toa.tif
/adapt/nobackup/projects/3sl/data/VHR/CAS/M1BS/WV02_20190207_M1BS_103001008ACFCA00-toa.tif
/adapt/nobackup/projects/3sl/data/VHR/CAS/M1BS/WV02_20110130_M1BS_103001000899D600-toa.tif
/adapt/nobackup/projects/3sl/data/VHR/CAS/M1BS/WV02_20181217_M1BS_1030010089CC6D00-toa.tif
/adapt/nobackup/projects/3sl/data/VHR/CAS/M1BS/WV02_20131206_M1BS_10300100291F4300-toa.tif
/adapt/nobackup/projects/3sl/data/VHR/CAS/M1BS/WV03_20160307_M1BS_10400100196AFB00-toa.tif
/adapt/nobackup/projects/3sl/data/VHR/CAS/M1BS/WV03_20210209_M1BS_104001006477DB00-toa.tif
/adapt/nobackup/projects/3sl/data/VHR/CAS/M1BS/WV03_20160123_M1BS_1040010018A59100-toa.tif
/adapt/nobackup/projects/3sl/data/VHR/CAS/M1BS/WV03_2015032

### Footprint this list

In [11]:
import sys
sys.path.append('/efs/pmontesa/code/geoscitools')
sys.path.append('/home/pmontesa/code/geoscitools')
import maplib
import footprintlib
from footprintlib import *

r_fn_list = list_strips_tap_toa
f_gdf_lst = [raster_footprint(r_fn, DO_DATAMASK=False, GET_ONLY_DATASETMASK=False, R_READ_MODE='r', MANY_CRS=True) for r_fn in r_fn_list]

In [18]:
outprint_fn = '/adapt/nobackup/people/pmontesa/userfs02/_share/Senegal_LCLUC/footprint_strips_tap_toa.gpkg'
# Build a footprint database geodataframe from all rasters
footprint_gdf = build_footprint_db(f_gdf_lst, TO_GCS=False, WRITE_GPKG=True, OUT_F_NAME=outprint_fn, OUT_LYR_NAME='toa')
footprint_gdf['footprint_name'] = footprint_gdf['file'].str.split('_', expand=True)[0]

footprint_gdf['batch_name'] = footprint_gdf['path'].apply(lambda x: os.path.splitext(os.path.basename(x))[0])


Building GDF from list...
Wrote out spatial footprints to /adapt/nobackup/people/pmontesa/userfs02/_share/Senegal_LCLUC/footprint_strips_tap_toa.gpkg


In [16]:
footprint_gdf.head()

,geometry,footprint_name,path,file,area_km2,area_ha,batch_name
0,"POLYGON ((-14.46377 12.46407, -14.46258 13.025...",WV03,/adapt/nobackup/projects/3sl/data/VHR/CAS/M1BS,WV03_20170226_M1BS_104001002802FF00-toa.tif,994.541200,99454.1200,M1BS
1,"POLYGON ((-14.50624 12.95314, -14.50510 13.515...",WV02,/adapt/nobackup/projects/3sl/data/VHR/CAS/M1BS,WV02_20170126_M1BS_1030010062B12000-toa.tif,1267.819200,126781.9200,M1BS
2,"POLYGON ((-13.65071 12.95832, -13.64655 13.707...",WV02,/adapt/nobackup/projects/3sl/data/VHR/CAS/M1BS,WV02_20190207_M1BS_103001008ACFCA00-toa.tif,1721.166944,172116.6944,M1BS
3,"POLYGON ((-14.12320 12.54020, -14.12150 13.031...",WV02,/adapt/nobackup/projects/3sl/data/VHR/CAS/M1BS,WV02_20110130_M1BS_103001000899D600-toa.tif,1030.705628,103070.5628,M1BS
4,"POLYGON ((-13.93051 12.95582, -13.92834 13.455...",WV02,/adapt/nobackup/projects/3sl/data/VHR/CAS/M1BS,WV02_20181217_M1BS_1030010089CC6D00-toa.tif,1156.328908,115632.8908,M1BS
